In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Global\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Global\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Global\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Load the dataset (replace the file paths with your dataset location)
train_data = pd.read_csv("new_train_data_s140.csv", encoding='ISO-8859-1')
test_data = pd.read_csv("new_test_data_s140.csv", encoding='ISO-8859-1')
# Rename columns for easier access
columns = ['Polarity', 'Id', 'Date', 'Query', 'User', 'Text']
train_data.columns = columns
test_data.columns = columns
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# Drop irrelevant columns
combined_data = combined_data.drop(['Id', 'Date', 'Query', 'User'], axis=1)
# Check the shape of the combined dataset
print("Combined Data Shape:", combined_data.shape)

# Check for null values
print("Null values in Train Data:\n", train_data.isnull().sum())
print("Null values in Test Data:\n", test_data.isnull().sum())

Combined Data Shape: (1600494, 2)
Null values in Train Data:
 Polarity    0
Id          0
Date        0
Query       0
User        0
Text        0
dtype: int64
Null values in Test Data:
 Polarity    0
Id          0
Date        0
Query       0
User        0
Text        0
dtype: int64


In [3]:
# Clean text function
def clean_text(text):
    # Remove URLs, mentions, hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^A-Za-z\s]', '', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization (reduce words to their base form)
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    # Join the tokens back into a single string
    clean_text = ' '.join(lemmatized_text)
    return clean_text

# Apply text cleaning to both training and testing datasets
train_data['Clean_Text'] = train_data['Text'].apply(clean_text)
test_data['Clean_Text'] = test_data['Text'].apply(clean_text)

# Check the cleaned data
print(train_data[['Text', 'Clean_Text']].head())

                                                Text  \
0  @Kenichan I dived many times for the ball. Man...   
1    my whole body feels itchy and like its on fire    
2  @nationwideclass no, it's not behaving at all....   
3                      @Kwesidei not the whole crew    
4                                        Need a hug    

                                        Clean_Text  
0  dived many time ball managed save rest go bound  
1                  whole body feel itchy like fire  
2                         behaving im mad cant see  
3                                       whole crew  
4                                         need hug  


In [4]:
# Split the dataset for training
X = train_data['Clean_Text']  # Features (cleaned text)
y = train_data['Polarity']    # Target (sentiment polarity)

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature extraction using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # Limit the number of features for simplicity

# Fit the vectorizer on training data and transform both train and validation sets
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# Check the TF-IDF matrix
print("TF-IDF train matrix shape:", X_train_tfidf.shape)
print("TF-IDF validation matrix shape:", X_val_tfidf.shape)

TF-IDF train matrix shape: (1279998, 5000)
TF-IDF validation matrix shape: (320000, 5000)


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

C:\Users\Global\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [6]:
import pickle

# Save the trained model to a file
with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the TF-IDF vectorizer to a file
with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Model and vectorizer saved successfully.")

Model and vectorizer saved successfully.


In [7]:
import pickle

# Load the trained model and vectorizer
model = pickle.load(open('logistic_regression_model.pkl', 'rb'))
vectorizer = pickle.load(open('tfidf_vectorizer.pkl', 'rb'))

In [8]:
import os
print(os.getcwd())

C:\Users\Global\Downloads\Data Mining Project
